In [1]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import copy

# Try all steve moon walks 

In [2]:
from glob import glob

# change to your own data path
steves = glob("./steve_Moonwalk/*.npz")

# Geo Objs

In [3]:
# Python Data
s = steves[0]

data = np.load(s)
# print(s)
# data_keys = data.keys()
# print(list(data_keys))

VIS_OFFSET_SCALE = 0.4
xyz1 = data['s_pc']
xyz2 = data['t_pc']
trans = data['trans'].flatten()
rot = data['rot']
xyz2 = (xyz2 - VIS_OFFSET_SCALE*trans)@rot

matching = data['correspondences']

# Point Cloud
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(xyz1)
pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(xyz2)


# matching Lines
SAMPLE_RATE = 0.002
n_match = len(matching)
sample_idc = np.random.choice(range(n_match), int(n_match*SAMPLE_RATE))
matching_sample = matching[sample_idc]

lines = o3d.geometry.LineSet.create_from_point_cloud_correspondences(pcd1, pcd2, matching_sample)


# vis_and_save([pcd1], 'pcd.png')

In [4]:
# def my_vis(*geo_objs, default_rotation=None, callback_func=None, save=False, path=None):
#     vis = o3d.visualization.Visualizer()
#     vis.create_window()
#     for obj in geo_objs:  
#         vis.add_geometry(obj)
    
#     if default_rotation:
#         ctr = vis.get_view_control()
#         ctr.rotate(*default_rotation)
    
#     vis.register_animation_callback(callback_func)
#     vis.run()
    
#     if (callback_func==None) and save:
#         vis.capture_screen_image(path,  do_render=True)

#     return vis

In [6]:
def rotate_view(vis):
    ctr = vis.get_view_control()
    ctr.rotate(0.2, 0.0)
    return False

In [6]:
# visualizer = my_vis(pcd1, pcd2, lines, default_rotation=(0,1000), callback_func=rotate_view)

In [7]:
import subprocess
TOO_LITTLE_MSG = """display dialog "Please pick at least 3 points :)" with title "Oops!" with icon caution buttons {"Retry"}"""


# source: http://www.open3d.org/docs/release/tutorial/visualization/interactive_visualization.html?highlight=visualizer
# def demo_crop_geometry(pcd, vis=None):
#     print("Demo for manual geometry cropping")
#     print(
#         "1) Press 'Y' twice to align geometry with negative direction of y-axis"
#     )
#     print("2) Press 'K' to lock screen and to switch to selection mode")
#     print("3) Drag for rectangle selection,")
#     print("   or use ctrl + left click for polygon selection")
#     print("4) Press 'C' to get a selected geometry and to save it")
#     print("5) Press 'F' to switch to freeview mode")
#     vis.clear_geometries()
#     vis.add_geometry([pcd])
#     return vis

def custom_draw_geo(*geo_objs, with_edit=False, pick_point=False, callback_func=rotate_view):
    if with_edit:
        vis = o3d.visualization.VisualizerWithEditing()
    else: 
        vis = o3d.visualization.Visualizer()

    vis.create_window(width=3500, height=2600)
    for geo_obj in geo_objs:
        vis.add_geometry(geo_obj)
        
    ctr = vis.get_view_control()
    ctr.rotate(100, 1000)
    
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    
    vis.register_animation_callback(callback_func)
    vis.run()
    vis.destroy_window()
    
    if with_edit and pick_point:
        return vis.get_picked_points()

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    
    custom_draw_geo(source_temp, target_temp)
    


def pick_points(pcd):
    print("")
    print("1) Please pick at least three correspondences using [shift + left click]" )
    print("   Press [shift + right click] to undo point picking")
    print("2) After picking points, press 'Q' to close the window")
    
    return custom_draw_geo(pcd, with_edit=True, pick_point=True)



def demo_manual_registration(source, target):
    print("Demo for manual ICP")
    print("Visualization of two point clouds before manual alignment")
    
    draw_registration_result(source, target, np.identity(4))

    # pick points from two point clouds and builds correspondences
    source.paint_uniform_color([1, 0.706, 0])
    target.paint_uniform_color([0, 0.651, 0.929])
    while True: 
        picked_id_target = pick_points(target)
        if len(picked_id_target) >= 3: 
            break
        else: 
            subprocess.call("osascript -e '{}'".format(TOO_LITTLE_MSG), shell=True)
            continue
        
    while True:
        picked_id_source = pick_points(source)
        if len(picked_id_source) >= 3: 
            break
        else: 
            subprocess.call("osascript -e '{}'".format(TOO_LITTLE_MSG), shell=True)
            continue
    
    if len(picked_id_source) != len(picked_id_target): 
        nr = min(len(picked_id_target), len(picked_id_source))
        picked_id_target = picked_id_target[:nr]
        picked_id_source = picked_id_source[:nr]
        
    corr = np.zeros((len(picked_id_source), 2))
    corr[:, 0] = picked_id_source
    corr[:, 1] = picked_id_target

    # estimate rough transformation using correspondences
    print("Compute a rough transform using the correspondences given by user")
    p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint()
    trans_init = p2p.compute_transformation(source, target,
                                            o3d.utility.Vector2iVector(corr))
    # point-to-point ICP for refinement
    print("Perform point-to-point ICP refinement")
    threshold = 0.03  # 3cm distance threshold
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    draw_registration_result(source, target, reg_p2p.transformation)
    print("")

In [ ]:
while True:
    # _ = my_vis(pcd1, pcd2, lines, default_rotation=(0,1000), callback_func=rotate_view)
    demo_manual_registration(pcd1, pcd2)

Demo for manual ICP
Visualization of two point clouds before manual alignment

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #25 (-1.6, 0.94, 0.49) to add in queue.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #236 (-1.4, 0.8, 0.81) to add in queue.
[Open3D INFO] Picked point #2819 (-1.6, 0.48, 0.65) to add in queue.
[Open3D INFO] Picked point #7041 (-1.3, 0.11, 0.51) to add in queue.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #1966 (-0.92, 0.16, 0.48) to add in queue.

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] No

## Exportdefault_rotation

## Voxels

colors = np.tile([0.6, 0.6, 0.8],  (xyz1.shape[0], 1))
noises = 0.1*np.random.normal(size=(xyz1.shape[0], 1))
pcd1.colors = o3d.utility.Vector3dVector(colors+noises)
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd1, voxel_size=0.05)

my_vis(voxel_grid)